In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [37]:
quart = pd.read_excel('Обработано.xlsx')

In [38]:
quart['Телефон'] = quart['Телефон'].apply(str)


# Теперь ВУ

In [39]:
vu_telefons = pd.read_excel('VU_taxi_3_quart.xlsx')

In [40]:
phones = vu_telefons['Телефон'].tolist()

In [41]:
pattern = r'\(\d{2}\)'
pattern2 = r'[а-яА-Яa-zA-Z]{1}'

for i, j in enumerate(phones):
    if len([x for x in j if x.isdigit()]) == 10:
        phones[i] = '7' + ''.join([x for x in j if x.isdigit()])    
    if len([x for x in j if x.isdigit()]) == 11:
        temp_arr = ''.join([x for x in j if x.isdigit()])
        if temp_arr[0] == '8':
            phones[i] = '7' + temp_arr[1:]
        elif temp_arr[0] == '7':    
            phones[i] = temp_arr

for i, j in enumerate(phones):
    if re.search(pattern, j):
        if '(61)' in j:
            phones[i] = phones[i].replace('(61)', ', 321-61-61, ')
        elif  '(39)' in j:   
            phones[i] = phones[i].replace('(39)', ', 305-73-39, ')
        elif '(73)' in j:
            phones[i] = phones[i].replace('(73)', '')
            phones[i] += ' , 558-73-09 '
        elif ('21') in j:
            phones[i] = phones[i].replace('(21)', '(921)')
    phones[i] = ''.join([x if x not in ',;!:.\\/+' else ' ' for x in j])        
    phones[i] = phones[i].lstrip(' ')
    phones[i] = phones[i].rstrip(' ')
    if re.search(pattern2, phones[i]):
        phones[i] = re.sub(pattern2, '', phones[i])
    phones[i] = ''.join([x for x in phones[i] if x not in '()-']) 

In [42]:
phones2 = phones.copy()
phones2

['3688345 89219810994',
 '3688345 89219810994',
 '5326592 89219820363',
 '79219213121',
 '4607524',
 '79119688048',
 '79213155325',
 '79213155325',
 '89218808990   79119888823',
 '89218808990   79119888823',
 '89218808990   79119888823',
 '79217402776',
 '79217402776',
 '79217402776',
 '3953538  79817019764',
 '3953538  79817019764',
 '79213348707',
 '89213489995    89602801911 ',
 '4475724 9345274',
 '3418949 89500349028  89117419676  79531404745',
 '7921 7628030  4307516',
 '79119005812',
 '79213017775',
 '7220475  89214266254',
 '79214071763',
 '79219400992  5226868',
 '79219400992  5226868',
 '79213241112',
 '79602796755  7568364 89111758365',
 '79650017549',
 '9161002  79112914858',
 '9602273',
 '89112874768\r\n89112874768',
 '9219706153 3513518',
 '9040802  89219008662  89522301552           9008662',
 '79217677771',
 '79817400646',
 '79052078372',
 '89119830355     3018493',
 '89214252726     5989979',
 '9817409941  2480427',
 '89213950287  3944111',
 '79817440054',
 '7952369986

In [43]:
for i, j in enumerate(phones2):
    phones2[i] = j.split()
    
from functools import reduce
for i, j in enumerate(phones2):
    try:
        phones2[i].append(reduce(lambda x, y: x + y, [z for z in j if len(z) < 7]))
    except:
        continue
        
        
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if x != len(j) - 1:
            if len(y) == 4 and len(j[x + 1]) == 7 and y[:2] in ('79', '89'):
                    phones2[i].append(y + j[x + 1])
            elif y in ('7', '8') and len(j[x + 1]) == 10:
                phones2[i].append(y + j[x + 1])
                
                
for i, j in enumerate(phones2):
    phones2[i] = [x for x in phones2[i] if len(x) >= 7]
    
    
    
    
for i, j in enumerate(phones2):
    for x, y in enumerate(j):
        if len(y) == 7:
            phones2[i][x] = '7812' + phones2[i][x]
        elif len(y) == 11 and y[0] == '8':
            phones2[i][x] = '7' + y[1:]
        elif len(y) == 10:
            phones2[i][x] = '7' + y
            
            
vu_telefons['Чистые_телефоны'] = phones2
vu_telefons['Чистые_телефоны']




0       [78123688345, 79219810994]
1       [78123688345, 79219810994]
2       [78125326592, 79219820363]
3                    [79219213121]
4                    [78124607524]
                   ...            
2595                            []
2596                 [79219034343]
2597                            []
2598    [79216513538, 79216513539]
2599                            []
Name: Чистые_телефоны, Length: 2600, dtype: object

In [44]:
lst_with_coincidenc = []
for index, row in enumerate(phones2):
    lst_with_coincidenc.append('')
    for i in quart['Телефон']:
        if str(i) in row:
            lst_with_coincidenc[index] = str(i)
            break
            
            
vu_telefons['Совпадение'] = lst_with_coincidenc  

In [45]:
vu_telefons_final = vu_telefons[vu_telefons['Совпадение'] != '']

In [46]:
vu_telefons_final = vu_telefons_final.merge(quart, left_on='Совпадение', right_on='Телефон', how='left')

In [47]:
vu_telefons_final = vu_telefons_final.fillna('')

In [48]:
vu_telefons_final['Дата приема'] = pd.to_datetime(vu_telefons_final['Дата приема'])

In [49]:
vu_telefons_final['Сравнение дат'] = vu_telefons_final['Дата приема'] >= vu_telefons_final['Дата']

In [50]:
vu_telefons_final

,Номер карты,Пациент,Дата приема,Дата первого посещения Пациента,Специализация приема,Телефон_x,Категория клиента,Sum([Стоимость полная]),Sum([Количество]),Чистые_телефоны,Совпадение,Дата,Тип,Телефон_y,Сравнение дат
0,77003,Беляева Елизавета Ивановна,2020-07-03,2006-12-22 00:00:00,Медтакси,8-921-740-27-76 Георгий!!!!,НАЛ,2500.0,1,[79217402776],79217402776,2020-07-03,Звонок,79217402776,True
1,77003,Беляева Елизавета Ивановна,2020-07-12,2006-12-22 00:00:00,Медтакси,8-921-740-27-76 Георгий!!!!,НАЛ,4500.0,1,[79217402776],79217402776,2020-07-03,Звонок,79217402776,True
2,77003,Беляева Елизавета Ивановна,2020-07-15,2006-12-22 00:00:00,Медтакси,8-921-740-27-76 Георгий!!!!,НАЛ,5100.0,1,[79217402776],79217402776,2020-07-03,Звонок,79217402776,True
3,134038,Удовиченко Зоя Никитична,2020-07-16,2009-05-13 00:00:00,Медтакси,"+79602796755, 756-83-64,8911-17-58-365",НАЛ,6200.0,1,"[79602796755, 78127568364, 79111758365]",78127568364,2020-07-15,Звонок,78127568364,True
4,100156139,Луганская Лидия Моисеевна,2020-07-01,2019-12-23 00:00:00,Медтакси,", +7(911)227-08-83",НАЛ,3000.0,1,[79112270883],79112270883,2020-07-01,Звонок,79112270883,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,120566717,Саадуева Ирина Петровна,2020-09-17,,Транспортная служба,", +7(981)784-35-74",НАЛ,670.0,1,[79817843574],79817843574,2020-09-30,Заявка,79817843574,False
189,120566717,Саадуева Ирина Петровна,2020-08-23,,Транспортная служба,", +7(981)784-35-74",НАЛ,590.0,1,[79817843574],79817843574,2020-09-30,Заявка,79817843574,False
190,120566717,Саадуева Ирина Петровна,2020-08-26,,Транспортная служба,", +7(981)784-35-74",НАЛ,590.0,1,[79817843574],79817843574,2020-09-30,Заявка,79817843574,False
191,120566717,Саадуева Ирина Петровна,2020-09-06,,Транспортная служба,", +7(981)784-35-74",НАЛ,590.0,1,[79817843574],79817843574,2020-09-30,Заявка,79817843574,False


In [51]:
vu_telefons_final.to_excel('ВУ_обработано.xlsx')